In [1]:
import pandas as pd
import os
import logging
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import re

# Set up logging
logging.basicConfig(filename='error.log', level=logging.ERROR)

# Define constants
EXCEL_FILES = Path.cwd().glob('*.xlsx')
OUTPUT_DIRECTORY = 'parsed_files'

### NOTE this is not the optimum way to map the columns, but it works for the given data ###
COLUMN_MAPPING = {
    r".*\boil\b.*": 'Oil_Production',
    r".*\boilprod\b.*": 'Oil_Production',
    r".*\bapi\b.*": 'API',
    r".*\bdate\b.*": 'Date',
    r".*\bproddate\b.*": 'Date',
    'well\s?name': 'Well_Name',
    # Add more regex patterns and mappings as needed
}


def setup_directory(directory):
    """
    Create the output directory if it doesn't exist.

    Args:
    - directory: The name of the output directory.

    Returns:
    - None
    """
    if not os.path.exists(directory):
        os.makedirs(directory)


def load_data(file):
    """
    Load an Excel file into a DataFrame, handle unnamed columns, and merge multiple sheets.

    Args:
    - file: The Excel file path.

    Returns:
    - df: The DataFrame containing the parsed data.
    """
    df = pd.read_excel(file)

    if any("Unnamed" in name for name in df.columns):
        df.dropna(axis=0, how='all', inplace=True)
        df.columns = df.iloc[0].tolist()
        df = df[1:]
        df = df.reset_index(drop=True)

    df.dropna(axis=0, how='all', inplace=True)
    df.reset_index(drop=True, inplace=True)

    if len(pd.ExcelFile(file).sheet_names) > 1:
        df = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True)

    df['file_name'] = file.stem
    df.rename(columns=update_column_names, inplace=True)
    return df


def validate_data(df, file):
    """
    Validate the presence of required columns in the DataFrame.

    Args:
    - df: The DataFrame to validate.
    - file: The file object representing the Excel file.

    Returns:
    - bool: True if the data is valid, False otherwise.
    """
    required_columns = ['Oil_Production', 'Date', 'API']
    missing_columns = [col for col in required_columns if col not in df.columns]

    if missing_columns:
        reasons = [f"Missing '{col}' column" for col in missing_columns]
        error_message = f"Invalid file: {file.name}. Reasons: {', '.join(reasons)}"
        logging.error(error_message)
        return False
    return True


def update_column_names(column):
    """
    Update the column names based on the regex patterns defined in COLUMN_MAPPING.

    Args:
    - column: The original column name.

    Returns:
    - updated_column: The standardized column name.
    """
    for pattern, standardized_name in COLUMN_MAPPING.items():
        regex = re.compile(pattern, re.IGNORECASE)
        if regex.match(column):
            return standardized_name
    return column


def adjust_column_widths(filename):
    """
    Adjust the column widths in the specified Excel file to fit the cell data.

    Args:
    - filename: The name of the Excel file to adjust.

    Returns:
    - None
    """
    wb = load_workbook(filename)
    for sheet in wb.sheetnames:
        for column in wb[sheet].columns:
            max_length = 0
            column_letter = get_column_letter(column[0].column)
            column = [str(cell.value) for cell in column]
            for cell in column:
                try:
                    if len(cell) > max_length:
                        max_length = len(cell)
                except:
                    pass
            adjusted_width = (max_length + 2)
            wb[sheet].column_dimensions[column_letter].width = adjusted_width
    wb.save(filename)


def main():
    """
    The main function.

    """
    setup_directory(OUTPUT_DIRECTORY)

    df_list = []
    for file in EXCEL_FILES:
        df = load_data(file)
        if not validate_data(df, file):
            continue
        df_list.append(df)

    df = pd.concat(df_list, ignore_index=True)


    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
    df.drop(columns='Comments', inplace=True, errors='ignore')

    output_filename = f"{OUTPUT_DIRECTORY}/parsed_file.xlsx"
    df.to_excel(output_filename, index=False)

    adjust_column_widths(output_filename)


if __name__ == "__main__":
    main()
